In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/task-3/CDMC2021_IoTMalware_Train.label
/kaggle/input/task-3/CDMC2021_IoTMalware_Train.data


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
train_cmd = pd.read_csv("../input/task-3/CDMC2021_IoTMalware_Train.data",delimiter=None,names=['Commands'])
train_cmd.head()

,Commands
0,entry0 sym.__uClibc_main;sym._fp_out_narrow sy...
1,entry0 sym.__uClibc_main;sym.resolve_cnc_addr ...
2,entry0 fcn.0804fdeb;main fcn.0804d1a0 fcn.0804...
3,entry0 unk.0x4002ac sym.__uClibc_main;sym._fp_...
4,sym.__udivsi3 sym.__aeabi_idiv0;sym.__aeabi_ui...


In [4]:
train_lbl = pd.read_csv("../input/task-3/CDMC2021_IoTMalware_Train.label",delimiter=None,names=['Labels'])
train_lbl.head()

,Labels
0,1
1,1
2,1
3,1
4,1


In [5]:
train = pd.concat([train_cmd,train_lbl],axis=1)
del train_cmd
del train_lbl
train.head()

,Commands,Labels
0,entry0 sym.__uClibc_main;sym._fp_out_narrow sy...,1
1,entry0 sym.__uClibc_main;sym.resolve_cnc_addr ...,1
2,entry0 fcn.0804fdeb;main fcn.0804d1a0 fcn.0804...,1
3,entry0 unk.0x4002ac sym.__uClibc_main;sym._fp_...,1
4,sym.__udivsi3 sym.__aeabi_idiv0;sym.__aeabi_ui...,1


In [6]:
length = []
for i in range(16521):
  str1 = train.Commands[i]
  str1 = str1.replace(' ','-')
  str1 = str1.replace(';',' ')  
  train.Commands[i] = str1
  list1 = str1.split()
  length.append(len(list1))
train['Length'] = length

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
X , Y  = train.drop(['Labels'], axis=1).to_numpy() , train.Labels.to_numpy()
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X , Y,test_size=.2,random_state=42)

In [8]:
train_under_df=pd.concat([pd.DataFrame(X_train), pd.Series(y_train)],axis=1, ignore_index= True)
train_under_df.columns=['Commands', 'Length', 'Labels']
train_under_df.head()

,Commands,Length,Labels
0,entry0-unk.0x40095c-sym.imp.__uClibc_main sym....,4,0
1,sym.__udivsi3-sym.__aeabi_idiv0 sym.__aeabi_ui...,3,1
2,sym._fini-unk.0x4a54-pc sym.mod_status_plugin_...,3,0
3,sym.ip_nat_sip_expect-fcn.080000f0 sym.ip_nat_...,5,0
4,fcn.00010fa1-unk.0x111f6-unk.0x1142b-unk.0x118...,5,0


In [9]:
Label1 = train_under_df[train_under_df.Labels == 1]
Label0 = train_under_df[train_under_df.Labels == 0]

In [10]:
train_under_df.Labels.value_counts()

0    9003
1    4213
Name: Labels, dtype: int64

In [11]:
Label0_under = Label0.sample(n = len(Label1), replace = False, random_state = 27) 
downsampled = pd.concat([Label0_under, Label1])
y_train_down = downsampled.Labels

In [12]:
train_over_df=pd.concat([pd.DataFrame(X_train), pd.Series(y_train)],axis=1, ignore_index= True)
train_over_df.columns=['Commands', 'Length', 'Labels']
train_over_df.head()

,Commands,Length,Labels
0,entry0-unk.0x40095c-sym.imp.__uClibc_main sym....,4,0
1,sym.__udivsi3-sym.__aeabi_idiv0 sym.__aeabi_ui...,3,1
2,sym._fini-unk.0x4a54-pc sym.mod_status_plugin_...,3,0
3,sym.ip_nat_sip_expect-fcn.080000f0 sym.ip_nat_...,5,0
4,fcn.00010fa1-unk.0x111f6-unk.0x1142b-unk.0x118...,5,0


In [13]:
Label1_over = Label1.sample(n = len(Label0), replace = True, random_state = 27) 
upsampled = pd.concat([Label1_over, Label0])
y_train_up = upsampled.Labels

In [14]:
!pip install eli5

In [15]:
from sklearn.model_selection import KFold
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_log_error
import eli5
from tqdm import tqdm_notebook

In [16]:
def get_features(df):
    df[['Commands', 'Length']]=df[['Commands', 'Length']].astype(str)
    default_preprocessor = CountVectorizer().build_preprocessor()
    def build_preprocessor(field):
        field_idx = list(df.columns).index(field)
        return lambda x: default_preprocessor(x[field_idx])

    vectorizer = FeatureUnion([
        ('Length', CountVectorizer(
            token_pattern='\d+', 
            preprocessor=build_preprocessor('Length'))),
        ('Commands', TfidfVectorizer(
            ngram_range=(2, 5) , sublinear_tf= True,
            preprocessor=build_preprocessor('Commands'), )),
    ])
    x_train_fit = vectorizer.fit(df[['Commands', 'Length']].values)
    x_train_trans = vectorizer.transform(df[['Commands', 'Length']].values)
    
    return vectorizer, x_train_fit, x_train_trans

In [17]:
without_train = train_under_df

In [18]:
vectorizer_without, x_trainfit_without, x_train_trans_without = get_features(without_train)
# vectorizer_under, x_trainfit_under, x_train_trans_under = get_features(downsampled)
# vectorizer_up, x_trainfit_up, x_train_trans_up = get_features(upsampled)

In [19]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

In [20]:
models = [LogisticRegression()]

In [21]:
def cv_df_formation(x_val, y_val):
    from sklearn.model_selection import cross_val_score
    CV = 5
    cv_df = pd.DataFrame(index=range(CV * len(models)))

    entries = []
    for model in models:
      model_name = model.__class__.__name__
      accuracies = cross_val_score(model, x_val, y_val, scoring='accuracy', cv=CV)
      for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))

    cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
    return cv_df


In [22]:
cv_df_without = cv_df_formation(x_train_trans_without, y_train)

In [23]:
cv_df_without

,model_name,fold_idx,accuracy
0,LogisticRegression,0,0.936460
1,LogisticRegression,1,0.930760
2,LogisticRegression,2,0.937949
3,LogisticRegression,3,0.940598
4,LogisticRegression,4,0.937949
